# Personnel problem

A construction company's work schedule on a certain site requires the following number of skilled personnel, called steel erectors, in the months of March through August:

|Month   |Personnel|
|--------|---------|
|mar     |4        |
|apr     |6        |
|may     |7        |
|jun     |4        |
|jul     |6        |
|aug     |2        |
|sep     |3        |

Personnel work at the site on the monthly basis.

Suppose that three steel erectors are on the site in February and three steel erectors must be on site in September. The problem is to determine how many workers to have on site in each month in order to minimize costs, subject to the following conditions:
- Transfer costs:
Adding a worker to this site costs \\$100 per worker and redeploying
a worker to another site costs \\$160.
- Transfer rules:
The company can transfer no more than three workers at the
start of any month, and under a union agreement,
it can redeploy no more than one-third of the current workers
in any trade from a site at the end of any month.
- Shortage time and overtime:
The company incurs a cost of \\$200 per worker per month for
having a surplus of steel erectors on site and a cost of \\$200 per worker
per month for having a shortage of workers at the site
(which must be made up in overtime).
Overtime cannot exceed 25 percent of the regular work time.

### Formulate this problem as a shortest path problem and solve it.
Hint:  you may want to think about the nodes in your formulation being tuples of the form (people,month)

In [1]:
# Load the gams extension
%load_ext gams_magic

In [2]:
%%gams
set people /1*7/;
set month /feb, mar, apr, may, jun, jul, aug, sep/;

set nodes(people,month) /set.people.set.month/;
display nodes;

set nodesfeb(people,month) /set.people.feb/;
set nodesmar(people,month) /set.people.mar/;
set nodesapr(people,month) /set.people.apr/;
set nodesmay(people,month) /set.people.may/;
set nodesjun(people,month) /set.people.jun/;
set nodesjul(people,month) /set.people.jul/;
set nodesaug(people,month) /set.people.aug/;
set nodessep(people,month) /set.people.sep/;

alias(people,p1,p2);
alias(month,m1,m2);
alias(nodes,j,i);

set arcs(p1,m1,p2,m2) /
set.nodesfeb . set.nodesmar
set.nodesmar . set.nodesapr
set.nodesapr . set.nodesmay
set.nodesmay . set.nodesjun
set.nodesjun . set.nodesjul
set.nodesjul . set.nodesaug
set.nodesaug . set.nodessep/;

parameter supply(people,month) /3.feb 1, 3.sep -1/;

Parameter demand(month) /feb 3, mar 4, apr 6, may 7, jun 4, jul 6, aug 2, sep 3/;

arcs(p1, m1, p2, m2)$(demand(m2) > 1.25 * ord(p2)) = no;
arcs(p1, m1, p2, m2)$((ord(p2) < [2/3] * ord(p1)) or (ord(p2) > ord(p1) + 3) ) = no;

Parameter costs(p1,m1,p2,m2);
costs(p1,m1,p2,m2)$( arcs(p1,m1,p2,m2) and ord(p2) > ord(p1) ) = 100*( ord(p2) - ord(p1) );
costs(p1,m1,p2,m2)$( arcs(p1,m1,p2,m2) and ord(p1) > ord(p2) ) = 160*( ord(p1) - ord(p2) );
costs(p1,m1,p2,m2)$( arcs(p1,m1,p2,m2) and ord(p2) > demand(m2) ) = 200*( ord(p2) - demand(m2) );
costs(p1,m1,p2,m2)$( arcs(p1,m1,p2,m2) and demand(m2) > ord(p2) ) = 200*( demand(m2) - ord(p2) );

variables TotalCost;
nonnegative variable flow(p1,m1,p2,m2);

equations bal, obj;

bal(people,month).. supply(people,month) =e= sum(arcs(people,month,p2,m2), flow(people,month,p2,m2)) - sum(arcs(p1,m1,people,month), flow(p1,m1,people,month));

Obj.. totalCost =e= sum(arcs, flow(arcs) * costs(arcs));

Model short /all/;
Solve short using lp minimizing totalCost;


,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),1160.0,51,135,LP,CPLEX,0.015


In [3]:
%gams_pull -d flow
%gams_pull month
soln=flow.loc[flow['level'] > 0]
soln.columns = ['p1','m1','p2','m2','level','marginal','lower','upper','scale']
used = dict.fromkeys(month)
for index, row in soln.iterrows():
    used[row['m1']] = row['p1']
    used[row['m2']] = row['p2']
print('Steel erectors employed:\n', used)

Steel erectors employed:
 {'feb': '3', 'mar': '4', 'apr': '7', 'may': '7', 'jun': '5', 'jul': '6', 'aug': '4', 'sep': '3'}
